In [1]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import urllib
import collections
import math
import os
import random
import zipfile
import datetime as dt

import numpy as np
import tensorflow as tf
import pandas as pd
import csv, re, string, pickle

In [3]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
remove_after = r' as in | refers to | etc.| also written | also known | e\.g\.|; |–|—| that is | that is,'
split_by = r'<br\/>| \/ |\n| \\ | or '
acr_punc  = '!"\',.:;<=>?`#%()[]{}'
note_punc = '!"\',.:;<=>?`#%()[]{}'
peroid = r'\.\s+'

In [4]:
dfile='/home/rishubj/text/MIMIC-data/NOTEEVENTS.csv'
# chunksize = 10 ** 8
chunk = pd.read_csv(dfile)
chunk.head()


/home/rishubj/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [5]:
# chunk.to_csv('/home/rishubj/text/MIMIC-data/chunk0NOTEEVENTS.csv')

In [6]:
chunk.head()['TEXT'][2], chunk.head()['TEXT'][0].split()

('Admission Date:  [**2119-5-4**]              Discharge Date:   [**2119-5-25**]\n\n\nService: CARDIOTHORACIC\n\nAllergies:\nAmlodipine\n\nAttending:[**Last Name (NamePattern1) 1561**]\nChief Complaint:\n81 yo F smoker w/ COPD, severe TBM, s/p tracheobronchoplasty [**5-5**]\ns/p perc trach [**5-13**]\n\nMajor Surgical or Invasive Procedure:\nbronchoscopy 3/31,4/2,3,[**6-12**], [**5-17**], [**5-19**]\ns/p trachealplasty [**5-5**]\npercutaneous tracheostomy [**5-13**] after failed extubation\ndown size trach on [**5-25**] to size 6 cuffless\n\n\nHistory of Present Illness:\nThis 81 year old woman has a history of COPD. Over the past five\n\nyears she has had progressive difficulties with her breathing.\nIn\n[**2118-6-4**] she was admitted to [**Hospital1 18**] for respiratory failure\ndue\nto a COPD exacerbation. Due to persistent hypoxemia, she\nrequired\nintubation and a eventual bronchoscopy on [**2118-6-9**] revealed marked\n\nnarrowing of the airways on expiration consistent with\nt

In [7]:
with open('acros.pickle', 'rb') as handle:
    acro_dct = pickle.load(handle)

In [69]:
acro_dct

{'frda': ['friedreichs ataxia'],
 'tibc': ['total iron-binding capacity'],
 'crtx': ['chemoradiotherapy'],
 'nwb': ['non-weight-bearing'],
 'auc': ['area under the curve'],
 'hcc': ['hepatocellular carcinoma'],
 'spet': ['single-photon emission tomography'],
 'ali': ['acute lung injury', 'acute limb injury'],
 'spep': ['serum protein electrophoresis'],
 'aur': ['ear'],
 'alg': ['antilymphocytic globulin'],
 'spec': ['specimen'],
 'mlc': ['mixed lymphocyte culture'],
 'kshv': ['kaposis sarcoma-associated herpesvirus'],
 'ptc': ['percutaneous transhepatic cholangiography'],
 'alc': ['alco123'],
 'pta': ['percutaneous transluminal angioplasty',
  'prior to admission',
  'post-traumatic amnesia'],
 'cmo': ['comfort measures only'],
 'upo': ['until proven otherwise'],
 'upj': ['ureteropelvic junction'],
 'ascad': ['arteriosclerotic coronary artery disease'],
 'cme': ['continuing medical education'],
 'cmd': ['cystic medial degeneration'],
 'l/s': ['lecithin-to-sphingomyelin ratio'],
 'fhx':

In [8]:
def clean_line(line):
    return ' '.join(re.sub(r'\b\d+\b', ' number_phrase ',
        re.sub(r'[\[]\*\*.*?\*\*[\]]', ' redacted_phrase ', line.encode('ascii','ignore')).translate(None, note_punc).lower()).split())

In [9]:
chunk['TEXT'].apply(clean_line).to_csv("/home/rishubj/text/MIMIC-data/cleaned.csv", index=False)

/home/rishubj/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [10]:
flat_list = [item for sublist in acro_dct.values() for item in sublist]

In [11]:
inverted_dict = {}
for k,v in acro_dct.items():
    for vv in v:
        if vv not in inverted_dict:
            inverted_dict[vv] = []
        inverted_dict[vv].append(k)

In [13]:
inverted_dict['fever of unknown origin']

['eci', 'fuo']

In [14]:
pd.DataFrame(flat_list).to_csv('test.csv',encoding='utf-8-sig')

In [15]:
len(flat_list), len(set(flat_list))

(2727, 2611)

In [16]:
c = collections.Counter(flat_list)
p=sorted(c.items())

In [17]:
c['zinc deficiency']

1

In [18]:
i=100
p[i], c[p[i]]

(('alert and oriented', 2), 0)

In [19]:
# with open('/home/rishubj/text/code/test.csv', 'rb') as handle:
#     l= (handle.readlines()[0])
# chunk.head()['TEXT'][:30].apply(clean_line)[0]==l.strip()

In [20]:
# o=set()
# for line in chunk.head()['TEXT'][:30]:
#     l= (re.sub(r'\b\d+\b', ' number_phrase ',
#         re.sub(r'[\[]\*\*.*?\*\*[\]]', ' redacted_phrase ', line).translate(None, note_punc).lower()).split())
#     for i in l:
#         if i in acro_dct:
#             o.add(i)
# for i in sorted(o):
#     print i, acro_dct[i]

In [21]:
# len([j  for j in acro_dct.values() if len(j)>1]), len(acro_dct)

In [22]:
# with open('/home/rishubj/text/MIMIC-data/smallestNOTES.csv') as f:
#     l = [x.strip() for x in f.readlines()] #tf.compat.as_str(f.read()).split('\n')

In [23]:
# l[1]

In [24]:
dfile='/home/rishubj/text/MIMIC-data/cleaned.csv'
cleaned = pd.read_csv(dfile, header=None).dropna()

In [25]:
cleaned.ix[0,0], # chunk['TEXT'][0]

/home/rishubj/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


('admission date redacted_phrase discharge date redacted_phrase service addendum radiologic studies radiologic studies also included a chest ct which confirmed cavitary lesions in the left lung apex consistent with infectious process/tuberculosis this also moderate-sized left pleural effusion head ct head ct showed no intracranial hemorrhage or mass effect but old infarction consistent with past medical history abdominal ct abdominal ct showed lesions of t10 and sacrum most likely secondary to osteoporosis these can be followed by repeat imaging as an outpatient redacted_phrase redacted_phrase redacted_phrase md redacted_phrase dictated by redacted_phrase medquist36 d redacted_phrase number_phrase t redacted_phrase number_phrase job redacted_phrase',)

In [26]:
for sent in cleaned.iloc[:,0]:
    print (sent)
    break

admission date redacted_phrase discharge date redacted_phrase service addendum radiologic studies radiologic studies also included a chest ct which confirmed cavitary lesions in the left lung apex consistent with infectious process/tuberculosis this also moderate-sized left pleural effusion head ct head ct showed no intracranial hemorrhage or mass effect but old infarction consistent with past medical history abdominal ct abdominal ct showed lesions of t10 and sacrum most likely secondary to osteoporosis these can be followed by repeat imaging as an outpatient redacted_phrase redacted_phrase redacted_phrase md redacted_phrase dictated by redacted_phrase medquist36 d redacted_phrase number_phrase t redacted_phrase number_phrase job redacted_phrase


In [27]:
# all_words = [word for word in sent.split() for sent in cleaned.iloc[:,0] if len(word)>0]
all_words = [word for sent in cleaned.iloc[:,0] for word in sent.split() if len(word)>0]

In [28]:
len(all_words)

512392754

In [29]:
cleaned.iloc[104286], # chunk.ix[104286, "TEXT"], clean_line(chunk.ix[104286, "TEXT"])

(0    normal sinus rhythm q waves in leads v1-v4 sug...
 Name: 104287, dtype: object,)

In [30]:
cleaned.iloc[104284:104287]

,0
104284,sinus rhythm normal tracing compared to the pr...
104285,sinus rhythm with borderline sinus tachycardia...
104287,normal sinus rhythm q waves in leads v1-v4 sug...


In [31]:
for i, line in enumerate(cleaned.iloc[:,0]):
    if type(line)!=str:
        print (i,line)

In [32]:
# def build_dataset(all_words, lines, n_words):
#     """Process raw inputs into a dataset."""
#     count = [['UNK', -1]]
#     count.extend(collections.Counter(all_words).most_common(n_words - 1))
#     dictionary = dict()
#     for word, _ in count:
#         dictionary[word] = len(dictionary)
#     print ("made vocabulary")
#     data = list()
#     unk_count = 0
#     for line in lines:
#         line_data=[]
#         for word in line.split():
#             if len(word)>0:
#                 if word in dictionary:
#                     index = dictionary[word]
#                 else:
#                     index = 0  # dictionary['UNK']
#                     unk_count += 1
#                 line_data.append(index)
#         if len(line_data)>0:
#             data.append(line_data)
#     count[0][1] = unk_count
#     reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
#     return data, count, dictionary, reversed_dictionary


# # old real version:
# def build_dataset(all_words, lines, n_words):
#     """Process raw inputs into a dataset."""
#     count = [['UNK', -1]]
#     count.extend(collections.Counter(all_words).most_common(n_words - 1))
#     dictionary = dict()
#     for word, _ in count:
#         dictionary[word] = len(dictionary)
#     print ("made vocabulary")
#     data = list()
#     unk_count = 0
#     for word in all_words:
#         if word in dictionary:
#             index = dictionary[word]
#         else:
#             index = 0  # dictionary['UNK']
#             unk_count += 1
#         data.append(index)
#     count[0][1] = unk_count
#     reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
#     return data, count, dictionary, reversed_dictionary

In [33]:
# vocabulary_size = 10000
# data, count, dictionary, reverse_dictionary = build_dataset(all_words, cleaned.iloc[:,0], vocabulary_size)

In [34]:
count = [['UNK', -1]]
counter = collections.Counter(all_words)

In [35]:
len(counter)

1423648

In [36]:
# all_words_in_acro_dct = [word for defins in acro_dct.values() for sent in defins for word in sent.split() if len(word)>0]
# all_words_in_acro_dct.extend([word for word in acro_dct])
# print(len(all_words_in_acro_dct))
# all_words_in_acro_dct = set(all_words_in_acro_dct)
# print(len(all_words_in_acro_dct))

In [37]:
# i=0
# for word in all_words_in_acro_dct:
#     if word in counter:
#         i+=1
# #     else:
# #         print(word)
# print(i)

In [38]:
i=0
all_words_in_acro_dct = []

for word in acro_dct:
    if word in counter:
        all_words_in_acro_dct.append(word)
        all_words_in_acro_dct.extend([word for sent in acro_dct[word] for word in sent.split() if len(word)>0])
        i+=1
all_words_in_acro_dct = set(all_words_in_acro_dct)
print(i, len(all_words_in_acro_dct))

(1882, 4492)


In [39]:
b=counter.most_common(8000)
c=set(zip(*b)[0])
print (len(c), len(c-all_words_in_acro_dct))

(8000, 5907)


In [40]:

# real version:
def build_dataset(all_words, lines, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    b=counter.most_common(n_words)
    c=set(zip(*b)[0])
    print (len(c), len(c-all_words_in_acro_dct))
    # add 1 to all
    for word in c-all_words_in_acro_dct:
        count.append([word, counter[word]+1])
    for word in all_words_in_acro_dct:
        if word not in counter:
            val = 1
        else:
            val = counter[word]+1
        count.append([word, val])
            
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    print ("made vocabulary")
    data = list()
    unk_count = 1 # add 1 to all
    for word in all_words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
#     acro_dct_num = {}
#     for word in all_words_in_acro_dct:
#         defins = acro_dct[word]
#         acro_dct_num[dictionary[word]] = [[dictionary[w] for w in sent.split() if len(w)>0] for sent in defins]
    return data, count, dictionary, reversed_dictionary #, acro_dct_num

In [41]:
base_vocabulary_size = 8000
data, count, dictionary, reverse_dictionary = build_dataset(all_words, cleaned.iloc[:,0], base_vocabulary_size)

(8000, 5907)
made vocabulary


In [42]:
vocabulary_size = len(dictionary)

In [43]:
acro_dct_num = {}
for word, defins in acro_dct.items():
    if word in counter:
        acro_dct_num[dictionary[word]] = [[dictionary[w] for w in sent.split() if len(w)>0] for sent in defins]

In [44]:
len(dictionary), len(acro_dct_num), len(acro_dct)

(10400, 1882, 2169)

In [45]:
a=all_words_in_acro_dct-set(counter)

In [46]:
len(a)

203

In [47]:
len(counter), counter.most_common(3)

(1423648,
 [('number_phrase', 36378616),
  ('redacted_phrase', 12596126),
  ('and', 10367252)])

In [48]:
counter[acro_dct.keys()[-1]], dictionary[acro_dct.keys()[1]]

(14, 5912)

In [49]:
jcounts = collections.defaultdict(int)
for j in data:
    if j in acro_dct_num:
        jcounts[len(acro_dct_num[j])]+=1
jcounts

defaultdict(int,
            {1: 23389625,
             2: 35476688,
             3: 17040431,
             4: 3871066,
             5: 1275107,
             7: 373977})

In [50]:
# acro_dct_num[5913]

KeyError: 5913

In [51]:
i*1.0/len(data)

3.672963728132658e-06

In [52]:
np.arange(10)[5-4:5], np.arange(10)[5+1:5+5]

(array([1, 2, 3, 4]), array([6, 7, 8, 9]))

In [53]:
rcounts = collections.defaultdict(int)
k=0
l=0
for i,j in acro_dct.items():
    rcounts[len(j)]+=1
    if len(j)==1:
        k+=1
#         print (i,j)
        if i==j[0]:
            l+=1
            print (i,j)
    if len(j)>=5:
        print (i,j)
rcounts, k, l

('mr', ['medical representative', 'mitral regurgitation', 'modified release', 'mental retardation', 'menstrual regulation'])
('ic', ['immunocompromised', 'ileocecal', 'immune complex', 'informed consent', 'interstitial cystitis', 'intracardiac', 'intensive care'])
('rad', ['right axis deviation', 'reactive airway disease', 'reactive attachment disorder', 'radiation absorbed dose', 'reflex anal dilatation'])
('ws', ['werner syndrome', 'wolfram syndrome', 'warkany syndrome', 'water-soluble', 'williams syndrome', 'west syndrome', 'waardenburg syndrome'])
('ed', ['effective dose', 'emergency department', 'ectodermal dysplasia', 'erectile dysfunction', 'emotional distress'])
('pap', ['papanicolaou stain', 'pulmonary artery pressure', 'positive airway pressure', 'pulmonary alveolar proteinosis', 'papanicolaou test'])
('atcc', ['atcc'])
('cd', ['chemical dependency', 'celiac disease', 'crohns disease', 'cluster of differentiation', 'controlled delivery'])
('ap', ['apical', 'alkaline phosphata

(defaultdict(int, {1: 1775, 2: 288, 3: 70, 4: 22, 5: 10, 7: 4}), 1775, 1)

In [54]:
s=u'posterior\xe2\u20ac\u201canterior posteroanterior'
s.encode('ascii','ignore')

'posterioranterior posteroanterior'

In [55]:
all_words[:100]

['admission',
 'date',
 'redacted_phrase',
 'discharge',
 'date',
 'redacted_phrase',
 'service',
 'addendum',
 'radiologic',
 'studies',
 'radiologic',
 'studies',
 'also',
 'included',
 'a',
 'chest',
 'ct',
 'which',
 'confirmed',
 'cavitary',
 'lesions',
 'in',
 'the',
 'left',
 'lung',
 'apex',
 'consistent',
 'with',
 'infectious',
 'process/tuberculosis',
 'this',
 'also',
 'moderate-sized',
 'left',
 'pleural',
 'effusion',
 'head',
 'ct',
 'head',
 'ct',
 'showed',
 'no',
 'intracranial',
 'hemorrhage',
 'or',
 'mass',
 'effect',
 'but',
 'old',
 'infarction',
 'consistent',
 'with',
 'past',
 'medical',
 'history',
 'abdominal',
 'ct',
 'abdominal',
 'ct',
 'showed',
 'lesions',
 'of',
 't10',
 'and',
 'sacrum',
 'most',
 'likely',
 'secondary',
 'to',
 'osteoporosis',
 'these',
 'can',
 'be',
 'followed',
 'by',
 'repeat',
 'imaging',
 'as',
 'an',
 'outpatient',
 'redacted_phrase',
 'redacted_phrase',
 'redacted_phrase',
 'md',
 'redacted_phrase',
 'dictated',
 'by',
 'reda

In [56]:
collections.Counter(all_words[:1000]).most_common(vocabulary_size - 1)

[('number_phrase', 65),
 ('of', 39),
 ('the', 32),
 ('redacted_phrase', 22),
 ('to', 20),
 ('and', 19),
 ('a', 18),
 ('with', 18),
 ('in', 16),
 ('is', 13),
 ('on', 11),
 ('mg', 10),
 ('she', 9),
 ('was', 9),
 ('no', 9),
 ('as', 8),
 ('an', 7),
 ('qd', 7),
 ('at', 7),
 ('history', 6),
 ('are', 6),
 ('admission', 6),
 ('or', 6),
 ('date', 5),
 ('not', 5),
 ('predicted', 5),
 ('nebulizer', 5),
 ('ct', 5),
 ('has', 5),
 ('cpap', 5),
 ('by', 5),
 ('her', 5),
 ('this', 4),
 ('prior', 4),
 ('oxygen', 4),
 ('also', 4),
 ('chest', 4),
 ('normal', 4),
 ('been', 4),
 ('left', 4),
 ('treatment', 4),
 ('right', 4),
 ('patient', 4),
 ('pressure', 4),
 ('head', 4),
 ('evidence', 4),
 ('mild', 3),
 ('extremities', 3),
 ('consistent', 3),
 ('past', 3),
 ('rate', 3),
 ('shortness', 3),
 ('taper', 3),
 ('studies', 3),
 ('negative', 3),
 ('heart', 3),
 ('machine', 3),
 ('due', 3),
 ('showed', 3),
 ('most', 3),
 ('angina', 3),
 ('exam', 3),
 ('breath', 3),
 ('lower', 3),
 ('changes', 3),
 ('well', 3),
 ('

In [57]:

# data_index = 0
# line_index = 0
# # generate batch data
# def generate_batch(data, batch_size, num_skips, skip_window):
#     global data_index
#     assert batch_size % num_skips == 0
#     assert num_skips <= 2 * skip_window
#     batch = np.ndarray(shape=(batch_size), dtype=np.int32)
#     context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
#     span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
#     buffer = collections.deque(maxlen=span)
#     while len(buffer) < span:
#         buffer = collections.deque(maxlen=span)
#         for _ in range(span):
#             buffer.append(data[line_index][data_index])
#             data_index += 1
#             if data_index > len(data[line_index]):
#                 line_index += 1
#                 break
                
#     for i in range(batch_size // num_skips):
#         target = skip_window  # input word at the center of the buffer
#         targets_to_avoid = [skip_window]
#         for j in range(num_skips):
#             while target in targets_to_avoid:
#                 target = random.randint(0, span - 1)
#             targets_to_avoid.append(target)
#             batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
#             context[i * num_skips + j, 0] = buffer[target]  # these are the context words
#         buffer.append(data[data_index])
#         data_index = (data_index + 1) % len(data)
#     # Backtrack a little bit to avoid skipping words in the end of a batch
#     data_index = (data_index + len(data) - span) % len(data)
#     return batch, context


In [58]:
# OLD good:

# data_index = 0
# # generate batch data
# def generate_batch(data, batch_size, num_skips, skip_window):
#     global data_index
#     assert batch_size % num_skips == 0
#     assert num_skips <= 2 * skip_window
#     batch = np.ndarray(shape=(batch_size), dtype=np.int32)
#     context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
#     span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
#     buff = collections.deque(maxlen=span)
#     for _ in range(span):
#         buff.append(data[data_index])
#         data_index = (data_index + 1) % len(data)
#     for i in range(batch_size // num_skips):
#         target = skip_window  # input word at the center of the buff
#         targets_to_avoid = [skip_window]
#         for j in range(num_skips):
#             while target in targets_to_avoid:
#                 target = random.randint(0, span - 1)
#             targets_to_avoid.append(target)
#             batch[i * num_skips + j] = buff[skip_window]  # this is the input word
#             context[i * num_skips + j, 0] = buff[target]  # these are the context words
#         buff.append(data[data_index])
#         data_index = (data_index + 1) % len(data)
#     # Backtrack a little bit to avoid skipping words in the end of a batch
#     data_index = (data_index + len(data) - span) % len(data)
#     return batch, context





data_index = 0
# generate batch data
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    buff = collections.deque(maxlen=span)
    
    acro_data = []
    for _ in range(span):
        # TODO: make function:
        next_data = data[data_index]
        if (next_data in acro_dct_num):
            bef = data[data_index-2*skip_window:data_index]
            aft = data[data_index+1:data_index+2*skip_window+1]
            acro_replacements = [bef+mid+aft for mid in acro_dct_num[next_data]]
            acro_data.append(acro_replacements)
        buff.append(next_data)
            
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # input word at the center of the buff
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buff[skip_window]  # this is the input word
            context[i * num_skips + j, 0] = buff[target]  # these are the context words
            
        # TODO: make function:
        next_data = data[data_index]
        if (next_data in acro_dct_num):
            bef = data[data_index-2*skip_window:data_index]
            aft = data[data_index+1:data_index+2*skip_window+1]
            acro_replacements = [bef+mid+aft for mid in acro_dct_num[next_data]]
            acro_data.append(acro_replacements)
        buff.append(next_data)
            
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, context, acro_data


def generate_batch_from_acro_data(acro_data, batch_size, skip_window):
    data_index = 0 # this is a local variable
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    
    global_low = 2*skip_window
    global_high = len(acro_data)-2*skip_window - 1
    for i in range(batch_size):
        mid = data_index + skip_window  # input word at the center of the buff
        if mid >= global_low and mid <= global_high:
            low = data_index
            high = data_index + 2*skip_window
        else:
            low = max(global_low, data_index)
            high = min(global_high, data_index + 2*skip_window)
            
        target = mid
        targets_to_avoid = [target]
        while target in targets_to_avoid:
            target = random.randint(low, high)
            
        batch[i] = acro_data[mid]  # this is the input word
        context[i, 0] = acro_data[target]  # these are the context words
        
        data_index+=1
        if data_index >= len(acro_data) - 2*skip_window:
            data_index = 0
            
        
    return batch, context

In [59]:
skip_window=2
data_index=0
batch_size=10
acro_data = np.arange(10)+200
generate_batch_from_acro_data(acro_data, batch_size, skip_window)

(array([202, 203, 204, 205, 206, 207, 202, 203, 204, 205], dtype=int32),
 array([[204],
        [204],
        [202],
        [204],
        [205],
        [205],
        [204],
        [204],
        [206],
        [204]], dtype=int32))

In [60]:

# sess = tf.Session()
# elems = (np.array([[1], [2], [3]]), np.array([-1, 1, -1]))
# alternate = tf.map_fn(lambda x: x[0][0] * x[1], elems, dtype=tf.int64)

# with sess.as_default():
#     alternate.eval()



In [61]:
print ("collected data")
batch_size = 128
acro_batch_size = 32
embedding_size = 300  # Dimension of the embedding vector.
skip_window = 2       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    train_acro_inputs = tf.placeholder(tf.int32, shape=[None, acro_batch_size])
    train_acro_context = tf.placeholder(tf.int32, shape=[None, acro_batch_size, 1])

    # Look up embeddings for inputs.
    embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    acro_embed = tf.nn.embedding_lookup(embeddings, train_acro_inputs)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
            normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
            valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()



collected data
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [62]:

# num_steps = 100
# softmax_start_time = dt.datetime.now()
# run(graph, num_steps=num_steps)
# softmax_end_time = dt.datetime.now()
# print("Softmax method took {} minutes to run 100 iterations".format((softmax_end_time-softmax_start_time).total_seconds()))


with graph.as_default():

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    nce_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))
    
    def nce_loss_loop(labels=train_context, inputs=embed):
        return tf.reduce_mean(
            tf.nn.nce_loss(weights=nce_weights,
                           biases=nce_biases,
                           labels=labels,
                           inputs=inputs,
                           num_sampled=num_sampled,
                           num_classes=vocabulary_size))
    
    mapped = tf.map_fn(lambda x: nce_loss_loop(x[0], x[1]), (train_acro_context,acro_embed), dtype=tf.float32)
    acro_nce_loss = tf.reduce_min(mapped)
    
    
    
#     # Input data.
#     train_inputs1 = tf.placeholder(tf.int32, shape=[smaller_batch])
#     train_context1 = tf.placeholder(tf.int32, shape=[smaller_batch, 1])
    
#     train_inputs2 = tf.placeholder(tf.int32, shape=[smaller_batch])
#     train_context2 = tf.placeholder(tf.int32, shape=[smaller_batch, 1])
    
#     train_inputs3 = tf.placeholder(tf.int32, shape=[smaller_batch])
#     train_context3 = tf.placeholder(tf.int32, shape=[smaller_batch, 1])

#     # Look up embeddings for inputs.
#     embed1 = tf.nn.embedding_lookup(embeddings, train_inputs1)
#     embed2 = tf.nn.embedding_lookup(embeddings, train_inputs2)
#     embed3 = tf.nn.embedding_lookup(embeddings, train_inputs3)
    
#     nce_loss = tf.reduce_mean(
#         tf.nn.nce_loss(weights=nce_weights,
#                        biases=nce_biases,
#                        labels=train_context1,
#                        inputs=embed1,
#                        num_sampled=num_sampled,
#                        num_classes=vocabulary_size))
#     nce_loss = tf.reduce_mean(
#         tf.nn.nce_loss(weights=nce_weights,
#                        biases=nce_biases,
#                        labels=train_context2,
#                        inputs=embed2,
#                        num_sampled=num_sampled,
#                        num_classes=vocabulary_size))
#     nce_loss = tf.reduce_mean(
#         tf.nn.nce_loss(weights=nce_weights,
#                        biases=nce_biases,
#                        labels=train_context3,
#                        inputs=embed3,
#                        num_sampled=num_sampled,
#                        num_classes=vocabulary_size))
    
    

#     optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(nce_loss)
    optimizer = tf.train.AdamOptimizer().minimize(nce_loss)
    acro_optimizer = tf.train.GradientDescentOptimizer(.01).minimize(acro_nce_loss)

    # Add variable initializer.
    init = tf.global_variables_initializer()


In [70]:

# def run(graph, num_steps):
#     with tf.Session(graph=graph) as session:
#         # We must initialize all variables before we use them.
#         init.run()
#         print('Initialized')

#         average_loss = 0
#         for step in range(num_steps):
#             batch_inputs, batch_context = generate_batch(data,
#                     batch_size, num_skips, skip_window)
#             feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

#             # We perform one update step by evaluating the optimizer op (including it
#             # in the list of returned values for session.run()
#             _, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
#             average_loss += loss_val

#             if step % 2000 == 0:
#                 if step > 0:
#                     average_loss /= 2000
#                 # The average loss is an estimate of the loss over the last 2000 batches.
#                 print('Average loss at step ', step, ': ', average_loss)
#                 average_loss = 0

#             # Note that this is expensive (~20% slowdown if computed every 500 steps)
#             if step % 10000 == 0:
#                 sim = similarity.eval()
#                 for i in range(valid_size):
#                     valid_word = reverse_dictionary[valid_examples[i]]
#                     top_k = 8  # number of nearest neighbors
#                     nearest = (-sim[i, :]).argsort()[1:top_k + 1]
#                     log_str = 'Nearest to %s:' % valid_word
#                     print (len(nearest), sim.shape, nearest, sim)
#                     for k in range(top_k):
#                         close_word = reverse_dictionary[nearest[k]]
#                         log_str = '%s %s,' % (log_str, close_word)
#                     print(log_str)
#         final_embeddings = normalized_embeddings.eval()

        
# includes acro_data        
def run(graph, num_steps, prefix=''):
    
    with tf.Session(graph=graph) as session:
        saver = tf.train.Saver() 
        # We must initialize all variables before we use them.
        init.run()
        print('Initialized')

        average_loss = 0
        acro_average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_context, acro_data = generate_batch(data,
                    batch_size, num_skips, skip_window)
            
            
            
            feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

            # We perform one update step by evaluating the optimizer op (including it
            # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, nce_loss], feed_dict=feed_dict)
            average_loss += loss_val
            
            
            
            # Now, redo it for acronyms:
            for data_word in acro_data:
                acro_batch_inputs = []
                acro_batch_context = []
                for single_acro_data in data_word:
                    a,b = generate_batch_from_acro_data(single_acro_data,
                            acro_batch_size, skip_window)
                    acro_batch_inputs.append(a)
                    acro_batch_context.append(b)
                acro_feed_dict = {train_acro_inputs: acro_batch_inputs, train_acro_context: acro_batch_context}
                _, acro_loss_val = session.run([acro_optimizer, acro_nce_loss], feed_dict=acro_feed_dict)
                acro_average_loss += acro_loss_val

            
            
            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                    acro_average_loss /= 2000
                # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average regular loss at step ', step, ': ', average_loss)
                print('Average acro loss at step ', step, ': ', acro_average_loss)
                average_loss = 0
                acro_average_loss = 0
                
                saver.save(session, '/home/rishubj/text/code/chkp/'+prefix+'model-step-{}.chkp'.format(step))

            # Note that this is expensive (~20% slowdown if computed every 500 steps)
            if step % 10000 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = reverse_dictionary[valid_examples[i]]
                    top_k = 8  # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log_str = 'Nearest to %s:' % valid_word
#                     print (len(nearest), sim.shape, nearest, sim)
                    for k in range(top_k):
                        close_word = reverse_dictionary[nearest[k]]
                        log_str = '%s %s,' % (log_str, close_word)
                    print(log_str)
        final_embeddings = normalized_embeddings.eval()
        saver.save(session, '/home/rishubj/text/code/chkp/'+prefix+'model-final.chkp')


In [71]:

num_steps = 10000
nce_start_time = dt.datetime.now()
run(graph, num_steps, prefix='medical1')
nce_end_time = dt.datetime.now()
print("NCE method took {} seconds to run {} iterations".format((nce_end_time-nce_start_time).total_seconds(), num_steps))

Initialized
('Average regular loss at step ', 0, ': ', 259.7983093261719)
('Average acro loss at step ', 0, ': ', 921.0973663330078)
Nearest to contractility: participated, haemophilus, dated, gum, granulomas, repleting, po2, suture,
Nearest to cigarettes: estrogen, clotting, paralysis, language, swelling, vascularity, desitin, enhancement,
Nearest to travel: xanax, po433, intraosseous, digoxin, acei, bony, hypoglycemic, plts,
Nearest to foul: polyneuropathy, elevated, delivered, maes, ehec, sodium, dihydrotestosterone, iron,
Nearest to spoke: spike, electrocardiogram, respiratory, adls, almost, rib, leave, mvr,
Nearest to vanco: brings, chirurgiae, lungs, parameters, grams/hour, gall, mdct, tavr,
Nearest to chloride-: -40cc/hr, remeron, duodenum, eol, sharing, postpartum, verify, walker,
Nearest to glass: srom, rvt, limitations, sxa, cinv, cyclenonspecific, acid16, isolated,
Nearest to reassess: vai, roughly, smoker, reaction, seems, crest, pedi, standing,
Nearest to tired: legionella

In [75]:
prefix='medical_long-'

In [72]:

num_steps = 100000
nce_start_time = dt.datetime.now()
run(graph, num_steps, prefix=prefix)
nce_end_time = dt.datetime.now()
print("NCE method took {} seconds to run {} iterations".format((nce_end_time-nce_start_time).total_seconds(), num_steps))

Initialized
('Average regular loss at step ', 0, ': ', 228.44703674316406)
('Average acro loss at step ', 0, ': ', 2218.4244842529297)
Nearest to contractility: -60cc/hr, diameter, a/bs, mds, lotion, window, t1-weighted, deformities,
Nearest to cigarettes: cant, abx, cto, pft, tranexamic, end-systolic, nyd, gtts,
Nearest to travel: c3-, bruises, master, beneath, c5-c6, we, swallowing, dislocated,
Nearest to foul: fena, npt, second, tem, fluoroscopic, tcp, t-sicu, preventative,
Nearest to spoke: baths, seven, dilatation, ulceration, interventricular, mucous, distinct, losses,
Nearest to vanco: 7p-7a, routine-pending, thereafter, mechanically, friend, experienced, non-insulin-dependent, pneumoboots,
Nearest to chloride-: subq, sjogrens, nonketotic, happy, diminshed, kcal, bibasilar, perineum,
Nearest to glass: went, nitroblue, ig, epogen, vitals, tf150cc/kg/d, tnm, stabilization,
Nearest to reassess: nephrogenic, pathologic, renally, concurrent, overload, cholecalciferol, slowly, overnoc

('Average regular loss at step ', 36000, ': ', 2.319436468727887)
('Average acro loss at step ', 36000, ': ', 14.717608857301995)
('Average regular loss at step ', 38000, ': ', 2.3510603500977156)
('Average acro loss at step ', 38000, ': ', 14.485197513419203)
('Average regular loss at step ', 40000, ': ', 2.2355495259165763)
('Average acro loss at step ', 40000, ': ', 14.538003649514168)
Nearest to contractility: redacted_phrase, diameter, ems, again, 20mg, the, lotion, caltibc-,
Nearest to cigarettes: count-, arrhythmia, day, abx, totbili-, drip, docusate, alcohol,
Nearest to travel: redacted_phrase, number_phrase, UNK, brbpr, drugs, maintained, coumadin, currently,
Nearest to foul: grew, second, few, ointment, lactate-, UNK, arrange, gabapentin,
Nearest to spoke: seven, mg, UNK, asa-neg, redacted_phrase, multivitamins, baths, date/time,
Nearest to vanco: number_phrase, to, and, thereafter, experienced, lipitor, caltibc-, was,
Nearest to chloride-: sodium-, urea, hct-, sig, creat-, c

('Average regular loss at step ', 78000, ': ', 2.509492676138878)
('Average acro loss at step ', 78000, ': ', 14.177698557151016)
('Average regular loss at step ', 80000, ': ', 2.5141112948954105)
('Average acro loss at step ', 80000, ': ', 15.250934809332248)
Nearest to contractility: redacted_phrase, number_phrase, the, UNK, mg, and, she, mcg,
Nearest to cigarettes: redacted_phrase, UNK, alcohol, she, the, number_phrase, he, a,
Nearest to travel: UNK, redacted_phrase, number_phrase, of, or, no, on, he,
Nearest to foul: UNK, as, and, redacted_phrase, of, the, in, number_phrase,
Nearest to spoke: redacted_phrase, UNK, and, mg, to, in, was, as,
Nearest to vanco: UNK, and, number_phrase, redacted_phrase, to, with, was, vancomycin,
Nearest to chloride-: sodium-, co2-, hct-, *, refills*, na-, urea, creat-,
Nearest to glass: number_phrase, of, UNK, in, went, -, no, the,
Nearest to reassess: as, number_phrase, on, sig, po, initially, and, overload,
Nearest to tired: she, on, UNK, glucose-, *

In [64]:

num_steps = 1000
nce_start_time = dt.datetime.now()
run(graph, num_steps)
nce_end_time = dt.datetime.now()
print("NCE method took {} seconds to run {} iterations".format((nce_end_time-nce_start_time).total_seconds(), num_steps))

Initialized
('Average regular loss at step ', 0, ': ', 228.37425231933594)
('Average acro loss at step ', 0, ': ', 1931.6600189208984)
Nearest to contractility: target, privileges, dorsalis, hh, puva, vfib, nephrotoxins, processing,
Nearest to cigarettes: acromioclavicular, notified, md, essentially, cool, panretinal, oblique, redness,
Nearest to travel: sick, nrb, nka, rhabdo, secured, transoesophageal, intermittently, q-t,
Nearest to foul: complementary, sr, nicu, record, wwi, fibrinogen, develop, fap,
Nearest to spoke: 10pm, coordinated, -90s, bandemia, tvc, tu, mouth, final,
Nearest to vanco: restrictive, derived, oil, contract, ml, s1s2, etoposide, repeated,
Nearest to chloride-: injection, b/p, canula, disk, ostomy, urgent, admitting, bronchoalveolar,
Nearest to glass: wear, colon, guaic, vc, report, could, minimally, tell,
Nearest to reassess: cmo, cll, looking, reason, osmolarity, opacified, memory, ihc,
Nearest to tired: -follow, supraclavicular, retroperitoneal, tah, lenis, m

In [65]:

num_steps = 20000
nce_start_time = dt.datetime.now()
run(graph, num_steps)
nce_end_time = dt.datetime.now()
print("NCE method took {} seconds to run {} iterations".format((nce_end_time-nce_start_time).total_seconds(), num_steps))

Initialized
('Average regular loss at step ', 0, ': ', 238.11558532714844)
('Average acro loss at step ', 0, ': ', 1759.5611572265625)
Nearest to contractility: mental, erectile, ahr, etiologies, angiographic, dccv, gda, esp,
Nearest to cigarettes: unremarkable, optics, daptomycin, cyst, transitioned, gastroparesis, normalized, klonopin,
Nearest to travel: l, regional/global, willebrand, obsessive-compulsive, lpp, overall, hib, shx,
Nearest to foul: make, cruciate, abgs, uso, tacrolimus, levels, hydration, cerebrovascular,
Nearest to spoke: blastic, sars, interphalangeal, pressor, gestation, confirm, stopping, abdominial,
Nearest to vanco: sclerosing, androgen, abuse, cubic, psychiatry, showing, polymerase, sedating,
Nearest to chloride-: several, ovary, triplet, cholesterol, dos, 40cc/hr, reversed, paradoxical,
Nearest to glass: cycled, headings, piggyback, w/contrast, nafcillin, hcts, microscope, c5,
Nearest to reassess: cleft, anal, ddd, particle/crystal, new, agreed, amniotic, son,

In [76]:
def get_word_embedding(word):
    with tf.Session(graph=graph) as session:
        saver = tf.train.Saver() 
        saver.restore(session, '/home/rishubj/text/code/chkp/'+prefix+'model-final.chkp')

        return (tf.nn.embedding_lookup(embeddings, dictionary[word]).eval())
        

In [78]:
embs = list(map(get_word_embedding, ['spoke', 'thoracics', 'and', 'or']))
for i,emb1 in enumerate(embs):
    for j,emb2 in enumerate(embs):    
        if i>j:
            print (i,j,np.sum(np.square(emb1-emb2)))

INFO:tensorflow:Restoring parameters from /home/rishubj/text/code/chkp/medical_long-model-final.chkp
INFO:tensorflow:Restoring parameters from /home/rishubj/text/code/chkp/medical_long-model-final.chkp
INFO:tensorflow:Restoring parameters from /home/rishubj/text/code/chkp/medical_long-model-final.chkp
INFO:tensorflow:Restoring parameters from /home/rishubj/text/code/chkp/medical_long-model-final.chkp
(1, 0, 226.42609)
(2, 0, 112.057465)
(2, 1, 168.55225)
(3, 0, 155.83246)
(3, 1, 235.27625)
(3, 2, 70.01285)
